In [2]:
import ee
import geemap


In [3]:
ee.Authenticate()



In [ ]:
# 초기화
ee.Initialize()

def get_sentinel1_data(start_date, end_date, geometry):
    collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                  .filterBounds(geometry)
                  .filterDate(ee.Date(start_date), ee.Date(end_date))
                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                  .filter(ee.Filter.eq('instrumentMode', 'IW'))
                  .select(['VV', 'VH']))

    def add_vv_vh_diff(image):
        vv_vh_diff = image.select('VV').subtract(image.select('VH')).rename('VV-VH')
        return image.addBands(vv_vh_diff)

    collection_with_diff = collection.map(add_vv_vh_diff)
    return collection_with_diff

def get_wdpa_aoi(region_name):
    wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")
    region = wdpa.filter(ee.Filter.eq('NAME', region_name))
    if region.size().getInfo() == 0:
        raise ValueError(f"No regions found with the name {region_name}")
    return region.geometry()

def export_to_csv(image, aoi, filename):
    task = ee.batch.Export.table.toDrive(
        collection=ee.FeatureCollection(image.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=aoi,
            scale=100
        ).map(lambda key, value: ee.Feature(None, {key: value}))),
        description=filename,
        fileFormat='CSV'
    )
    task.start()
    print(f"Exporting {filename} to Google Drive...")

def main(region_name='태안해안', start_date='2020-01-01', end_date='2020-01-31'):
    try:
        aoi = get_wdpa_aoi(region_name)
    except ValueError as e:
        print(e)
        return

    sentinel_data = get_sentinel1_data(start_date, end_date, aoi)
    resampled_image = sentinel_data.first().resample('bilinear').reproject(
        crs=sentinel_data.first().projection(), scale=100
    )
    export_to_csv(resampled_image, aoi, 'sentinel1_data.csv')

if __name__ == '__main__':
    main('태안해안', '2020-01-01', '2020-01-31')
